### Import 'match_year_year+1.xlsx' data where 'flag' is not 0 and save it as 'df_year_year+1' DataFrame

In [1]:
import pandas as pd

years = range(2015, 2022)

for year in years:

    # File paths for the year1 and year2 matched data
    file_path = f'../YtoY_diff/match_{year}_{year+1}.xlsx'

    # Read the files into dataframes
    df = pd.read_excel(file_path)

    # Filter the data based on the condition where 'flag' column is not equal to 0
    globals()[f"df_{year}_{year+1}"] = df[df['flag'] != 0][[f'{year}_ID', f'{year+1}_ID']]

### Inner-join all 'df_year_year+1' DataFrame on IDs

In [2]:
years = range(2015, 2022)
df_names = []

for year in years:

    # Define the list of dataframe names
    df_names.append(f'df_{year}_{year+1}')

# Initialize the combined dataframe with the first dataframe
combined_df = globals()[df_names[0]]

# Iterate over the pairs of dataframes and perform the join operation
for df_name in df_names[1:]:
    # Get the dataframe for the current iteration
    current_df = globals()[df_name]
    
    # Determine the common column between the current dataframe and the combined dataframe
    common_column = list(set(combined_df.columns) & set(current_df.columns))[0]
    
    # Perform the join operation
    combined_df = pd.merge(combined_df, current_df, on=common_column, how='inner')

In [3]:
combined_df.to_excel('../YtoY_diff/combined_data.xlsx', index=False)

### Duplicate every row and replace 'E' with 'M'  
#### (Metadata only contains 'E', so this is done to be able to match easily with raw data)

In [4]:
# Create a new DataFrame to hold the modified data with M + E
modified_df = pd.DataFrame()

for column in combined_df.columns:
    # Extract the column name and IDs
    ids = combined_df[column].tolist()
    
    # Duplicate the IDs, replacing 'E' with 'M'
    modified_ids = [i[:-1] + 'M' for i in ids]
    
    # Combine original and modified IDs in a single list, interweaving them
    combined_ids = [val for pair in zip(ids, modified_ids) for val in pair]
    
    # Add the combined IDs to the new DataFrame
    modified_df[column] = combined_ids

# Display the first few rows of the modified DataFrame to verify changes
modified_df.to_csv('combined_id_E_M.csv', index=False)